In [ ]:
# %run E:\IITD\Depression-IITD\common.ipynb
# %run E:\IITD\Depression-IITD\preprocessing.ipynb

Variables

In [1]:
SCALINGS = 4e-4
SFREQ = 500
WINDOW_SIZE = 1
SHOW = False
FMIN = 0.01
FMAX = 45
FREQ_BANDS = {
    'delta': (0.01, 4),
    'theta': (4, 8),
    'alpha1': (8, 10),
    'alpha2': (10, 13),
    'beta': (13, 30),
    'gamma': (30, 45)
}
ALL_CHANNELS=[]
GROUP1 = 'PRE-ACTIVE'
GROUP2 = 'POST-ACTIVE'

Importing all packages

In [44]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy.fft import fft
import time
import mne
import matplotlib
import matplotlib.pyplot as plt
import os
from scipy.stats import kurtosis, zscore, ttest_ind
from mne.preprocessing import create_ecg_epochs, create_eog_epochs, read_ica
from mne.time_frequency import tfr_morlet, tfr_array_morlet, morlet, AverageTFR
from itertools import product
import pywt
from scipy.signal import hilbert
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_sensors_connectivity
import networkx as nx
import numpy as np 
import re
from scipy.cluster.hierarchy import dendrogram, linkage
from colabcode import ColabCode
from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import MultiNiftiLabelsMasker
from nilearn import datasets


matplotlib.use('TkAgg')

In [4]:
# ColabCode(port=10000)

## Reading the EEG data

In [3]:
# folder_path = os.getcwd()+'\\Depression-Sample-dataset-AIIMS\\'
# items = os.listdir(folder_path)
# active_or_sham_list = [item for item in items if os.path.isdir(os.path.join(folder_path, item))]
# for active_or_sham in active_or_sham_list:
#     patient_folder_path = os.path.join(folder_path, active_or_sham)
#     items = os.listdir(patient_folder_path)
#     patients_list = [item for item in items if os.path.isdir(os.path.join(patient_folder_path, item))]
#     patients_list = ['PreetiSingh', 'Hemlata', 'VinodKumarSharma']
#     for patient in patients_list:
#         pre_post_int_folder_path = os.path.join(patient_folder_path, patient)
#         items = os.listdir(pre_post_int_folder_path)
#         pre_post_int_list = [item for item in items if os.path.isdir(os.path.join(pre_post_int_folder_path, item))]
#         for var in pre_post_int_list:
#             if var=='Pre':
#                 pre_path = os.path.join(pre_post_int_folder_path, var)
#                 file_path = pre_path + '\\' + '20230718201550_Preeti singh_22.08.23-01_Eye Close' + '.easy'
#                 break # Remove for all pre, post and intervention for a patient
#         break # Remove for all patients in active or sham
#     break # Remove for both active and sham

Data analysis

In [4]:
# file_path = os.getcwd()+'\\Depression-Sample-dataset-AIIMS\\Active\\VinodKumarSharma\\pre\\20230829195416_VinodKumarSharma_25.9.23_01_Eye Close.easy'
def data_transformation_easy(file_path):
	df = pd.read_csv(file_path, sep='\t')
	channel_str='Channel 1:P8\
		Channel 2:T8\
		Channel 3:CP6\
		Channel 4:FC6\
		Channel 5:F8\
		Channel 6:F4\
		Channel 7:C4\
		Channel 8:P4\
		Channel 9:AF4\
		Channel 10:Fp2\
		Channel 11:Fp1\
		Channel 12:AF3\
		Channel 13:Fz\
		Channel 14:FC2\
		Channel 15:Cz\
		Channel 16:CP2\
		Channel 17:PO3\
		Channel 18:O1\
		Channel 19:Oz\
		Channel 20:O2\
		Channel 21:PO4\
		Channel 22:Pz\
		Channel 23:CP1\
		Channel 24:FC1\
		Channel 25:P3\
		Channel 26:C3\
		Channel 27:F3\
		Channel 28:F7\
		Channel 29:FC5\
		Channel 30:CP5\
		Channel 31:T7\
		Channel 32:P7'

	channel_names = re.findall(r'Channel \d+:(\w+)', channel_str)
	channel_names.append('ax')
	channel_names.append('ay')
	channel_names.append('az')
	channel_names.append('trigger')
	channel_names.append('timestamp(ms)')
	global ALL_CHANNELS
	ALL_CHANNELS = np.array(channel_names[:-5])
	df.columns=channel_names
	transposed_data=df.T
	ch_names = df.columns.tolist()[:-5]
	ch_types = ['eeg' for i in range(32)]
	info = mne.create_info(ch_names=ch_names,ch_types=ch_types, sfreq=500)
	raw = mne.io.RawArray(transposed_data.values[:-5,:]/1e9, info) # Example: 33129984 nV = 0.033129984 V = 33129.984000000004 uV
	return raw
# data_transformation_easy(file_path)

Set montage

In [5]:
def set_montage(raw):
    mont1020 = mne.channels.make_standard_montage('standard_1020')
    ind = [i for (i, channel) in enumerate(mont1020.ch_names) if channel in ALL_CHANNELS]
    mont1020_new = mont1020.copy()
    mont1020_new.ch_names = [mont1020.ch_names[x] for x in ind]
    kept_channel_info = [mont1020.dig[x+3] for x in ind]
    mont1020_new.dig = mont1020.dig[0:3]+kept_channel_info
    raw.set_montage(mont1020_new)
    return raw

Calculate absolute power

In [6]:
def calculate_absolute_power(raw_data):
    psds, freqs = mne.time_frequency.psd_array_welch(raw_data.get_data(), fmin=FMIN, fmax=FMAX, sfreq=SFREQ)
    absolute_powers = {}
    for band, (FMIN, FMAX) in FREQ_BANDS.items():
        idx_band = np.logical_and(freqs >= FMIN, freqs <= FMAX)
        absolute_power = np.trapz(psds[:, idx_band], dx=(freqs[1] - freqs[0]), axis=-1)
        absolute_powers[band] = absolute_power
    total_absolute_power = sum(absolute_powers.values())
    return total_absolute_power


#### Time amplitude plot

In [7]:
def time_amplitude(raw):
    fig = raw.plot(
        n_channels=32, 
        scalings=SCALINGS,
        show=SHOW
        )
    # fig.savefig(f'MNE-graphs/time-amplitude/{title}-EEG.png')

    print(raw.info)
    # TODO: Extract statistical features from time domain such as mean, median, variance, skewness, kurtosis

#### Power spectral density plot

In [8]:
def psd(raw):
    fig = raw.plot_psd(
        picks=raw.info['ch_names'], 
        show=SHOW)
    # fig.savefig(f'MNE-graphs/psd-frequency/{title}.png')

#### Wavelet plot

In [9]:
# def wavelet(raw1, raw2=None, title):
#     raw1_avg = np.mean(raw1.get_data(), axis=0)
#     t = np.arange(0, 150, 1/SFREQ)
#     ts = t[:-1]
#     wavelet = 'db13'
#     level = 5 # level of decomposition based on your signal characteristics

#     # coeffs_multi_channel = []
#     # for i in range(32): 
#     coeffs1 = pywt.wavedec(raw1_avg, wavelet, level=level)
#         # coeffs_multi_channel.append(coeffs)
    
#     plt.figure(figsize=(10, 6))

#     # raw1
#     plt.subplot(4, 1, 1)
#     plt.plot(ts, raw1_avg, label='xyz')
#     plt.title('Channel 1')
#     plt.xlabel('Time')
#     plt.ylabel('Amplitude')
#     plt.legend()

#     # raw1 coeffs
#     plt.subplot(4, 1, 2)
#     for i in range(level+1):
#         plt.plot(t, pywt.upcoef('a', coeffs1[i], wavelet, level=level)[:len(t)], label=f'Level {i}')
#     plt.title('DWT')
#     plt.xlabel('Time')
#     plt.ylabel('Amplitude')
#     plt.legend()

#     if raw2:
#         raw2_avg = np.mean(raw2.get_data(), axis=0)
#         coeffs2 = pywt.wavedec(raw2_avg, wavelet, level=level)

#         # raw2
#         plt.subplot(4, 1, 3)
#         plt.plot(t, raw2_avg, label='ddd')
#         plt.title('Channel 1')
#         plt.xlabel('Time')
#         plt.ylabel('Amplitude')
#         plt.legend()

#         # raw2 coeffs
#         plt.subplot(4, 1, 4)
#         for i in range(level+1):
#             plt.plot(t, pywt.upcoef('a', coeffs2[i], wavelet, level=level)[:len(t)], label=f'Level {i}')
#         plt.title('DWT')
#         plt.xlabel('Time')
#         plt.ylabel('Amplitude')
#         plt.legend()

#     plt.tight_layout()
#     plt.show()

Band pass filtering

In [10]:
def band_pass_filter(raw, l_freq=FMIN, h_freq=FMAX):
    raw.filter(method= 'fir',
        phase= 'minimum',
        fir_window= 'hann',
        l_freq= l_freq,
        h_freq= h_freq)
    return raw

Rereferencing

Calculates the mean voltage from all electrodes at each time point and subtracts this mean from the voltage at each individual electrode.

In [11]:
def rereferencing(raw):
    raw.set_eeg_reference('average', projection=True).apply_proj() 
    return raw

Artifact Rejection (EOG/ECG) using Wavelet decomposition

In [12]:
# def wavelet_decompose(raw):
#     info = raw.info
#     coeffs = pywt.wavedec(raw.get_data(), 'db13', level=4)
#     threshold = 0.00001 # applying a shrinkage function that smoothly brings coefficients below the threshold to zero
#     coeffs_thresholded = [pywt.threshold(c, threshold, mode='soft') for c in coeffs] 
#     denoised_signal = pywt.waverec(coeffs_thresholded, 'db13')
#     raw = mne.io.RawArray(denoised_signal, info)
#     return raw

Crop signal

In [13]:
def crop(raw, l, h):
    return raw.crop(l,h)

Resampling

In [14]:
def resampling(raw, SFREQ=SFREQ):
    return raw.resample(sfreq=SFREQ)

Drop channels

Dropping extra channels in both groups
TODO: Instead of dropping we can convert 10-10 64 channels montage to 10-20 32 channels

In [15]:
def drop_channels(raw, raw_1):
    drop = []
    for chan in raw.info['ch_names']:
        if chan not in raw_1.info['ch_names']:
            drop.append(chan)
    raw.drop_channels(drop)
    return raw

Group 1 preprocessing

In [16]:
def g1_preprocess(raw):
    raw = set_montage(raw)
    raw  = band_pass_filter(raw, l_freq = FMIN, h_freq = FMAX)
    raw = rereferencing(raw)
    # raw = wavelet_decompose(raw)
    raw.info['bads'] = []
    return raw

Group 2 preprcessing

In [17]:
def g2_preprocessing(raw, raw_1, crop=True):
    # For epochs crop=False
    raw = drop_channels(raw, raw_1)
    if crop:
        raw = set_montage(raw)
        raw = raw.crop(50, 200)
    raw = resampling(raw)
    raw = band_pass_filter(raw, l_freq = 0.01, h_freq = 45)
    # raw = wavelet_decompose(raw)
    raw.info['bads'] = []
    return raw

Calculate average components of a graph

In [18]:
def calculate_avergae_components(G):
    connected_components = list(nx.connected_components(G))
    component_avg_lengths = []
    component_nbc_values = []
    component_eglo_values = []
    component_cc_values = []
    component_eloc_values = []

    for component in connected_components:
        subgraph = G.subgraph(component)
        component_avg_lengths.append(nx.average_shortest_path_length(subgraph))
        component_nbc_values.append(nx.betweenness_centrality(subgraph))
        component_eglo_values.append(nx.global_efficiency(subgraph))
        component_cc_values.append(nx.average_clustering(subgraph))
        component_eloc_values.append(nx.local_efficiency(subgraph))

    merged_nbc_values = {}
    for dnbc in component_nbc_values:
        merged_nbc_values.update(dnbc)
    merged_nbc_values = dict(sorted(merged_nbc_values.items(), key=lambda item: item[0]))
 
    overall_avg_length = sum(component_avg_lengths) / len(component_avg_lengths)
    overall_eglo_values = sum(component_eglo_values) / len(component_eglo_values)
    overall_cc_values = sum(component_cc_values) / len(component_cc_values)
    overall_eloc_values = sum(component_eloc_values) / len(component_eloc_values)

    return [overall_avg_length, merged_nbc_values, overall_eglo_values, overall_cc_values, overall_eloc_values]

## LOAD DATA

In [19]:
# # PRE VS CONTROL
# if GROUP1 == 'PRE-ALL' and GROUP2 == 'CONTROL':
    # raw_1 = mne.io.read_raw_fif("loadData/rawMDDpre5.fif")
    # raw_2 = mne.io.read_raw_fif("loadData/rawControlOnline104.fif")

    # epochs_1 = mne.read_epochs("loadData/epochsMDDpre5.fif", preload=False)
    # epochs_2 = mne.read_epochs("loadData/epochsControlOnline104.fif", preload=False)

In [20]:
# PRE VS POST - ACTIVE
if GROUP1 == 'PRE-ACTIVE' and GROUP2 == 'POST-ACTIVE':
    raw_1 = mne.io.read_raw_fif("loadData/rawMDDpre3Active.fif")
    raw_2 = mne.io.read_raw_fif("loadData/rawMDDpost3Active.fif")

    epochs_1 = mne.read_epochs("loadData/epochsMDDpre3Active.fif")
    epochs_2 = mne.read_epochs("loadData/epochsMDDpost3Active.fif")


Opening raw data file loadData/rawMDDpre3Active.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 32) active
    Range : 0 ... 74998 =      0.000 ...   149.996 secs
Ready.
Opening raw data file loadData/rawMDDpost3Active.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 32) active
    Range : 0 ... 74998 =      0.000 ...   149.996 secs
Ready.
Reading /home/vishwani/Downloads/IITD/Depression-IITD/loadData/epochsMDDpre3Active.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 32) active
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
73 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading /home/vishwani/Downloads/IITD/Depression-IITD/loadData/epochsMDDpost3Active.fif ...
    Read a total of 1 projection items:
        

/tmp/ipykernel_10587/1281472082.py:3: RuntimeWarning: This filename (loadData/rawMDDpre3Active.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_1 = mne.io.read_raw_fif("loadData/rawMDDpre3Active.fif")
/tmp/ipykernel_10587/1281472082.py:4: RuntimeWarning: This filename (loadData/rawMDDpost3Active.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_2 = mne.io.read_raw_fif("loadData/rawMDDpost3Active.fif")
/tmp/ipykernel_10587/1281472082.py:6: RuntimeWarning: This filename (loadData/epochsMDDpre3Active.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _ep

In [ ]:
# # POST -Active vs Control
# if GROUP1 == 'POST-ACTIVE' and GROUP2 == 'CONTROL':
    # raw_1 = mne.io.read_raw_fif("loadData/rawMDDpost3Active.fif")
    # raw_2 = mne.io.read_raw_fif("loadData/rawControlOnline104.fif")

    # epochs_1 = mne.read_epochs("loadData/epochsMDDpre3Active.fif", preload=False)
    # epochs_2 = mne.read_epochs("loadData/epochsControlOnline104.fif", preload=False)

# ELSE COMPUTE DATA

MDD data

In [ ]:
eye_close_1 = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', 'Active', 'PreetiSingh', 'Pre', '20230718201550_Preeti singh_22.08.23-01_Eye Close.easy' )
eye_close_2 = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', 'Active', 'Hemlata', 'Pre', '20230831105330_Hemlata_05.10.23_01_Eye Close.easy') 
eye_close_3 = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', 'Active', 'VinodKumarSharma', 'Pre', '20230829195416_VinodKumarSharma_25.9.23_01_Eye Close.easy' ) 
# eye_close_4 = '\\Depression-Sample-dataset-AIIMS\\Sham\\JitenderKumar\\pre\\' + '20230825020227_JitenderKumar_29.08.23_01_Eye Close'
# eye_close_5 = '\\Depression-Sample-dataset-AIIMS\\Sham\\SeemaKumari\\pre\\' + '20230827073914_SeemaKumari_11.09.23_01_Eye Close'
# eye_close_6 = '\\Depression-Sample-dataset-AIIMS\\Active\\PreetiSingh\\post\\' + '20230826225729_PreetiSingh_08.09.23_20_Eye Close'
# eye_close_7 = '\\Depression-Sample-dataset-AIIMS\\Active\\Hemlata\\post\\' + '20231019110530_Hemlata_19.10.23_19A_Eye Close'
# eye_close_8 = '\\Depression-Sample-dataset-AIIMS\\Active\\VinodKumarSharma\\post\\' + '20230901112451_VinodKumarSharma_11.10.23_20_Eye Close'
# eye_close_9 = '\\Depression-Sample-dataset-AIIMS\\Sham\\JitenderKumar\\post\\' + '20230827174115_JitenderKumar_13.09.23_20_Eye Close'
# eye_close_10 = '\\Depression-Sample-dataset-AIIMS\\Sham\\SeemaKumari\\post\\' + '20230829172634_SeemaKumari_23.9.23_20_Eye Close'
eye_close_MDD = [eye_close_1, eye_close_2, eye_close_3]

In [ ]:
raw_1 = []
for file_path in eye_close_MDD:
    raw = data_transformation_easy(file_path)
    # raw = mne.io.read_raw_edf(file, eog=None, misc=None, stim_channel='auto', exclude=(), infer_types=False, include=None, preload=True, units=None, encoding='utf8', verbose=None)
    # By default read_raw_edf convert units to V
    raw = g1_preprocess(raw)
    raw_1.append(raw.get_data())

In [ ]:
# M1 : Averaging using mean:

raw_1_avg = np.mean(raw_1, axis=0)
print(raw_1_avg)

del raw_1[:]
del raw_1
del eye_close_MDD[:]
del eye_close_MDD

In [ ]:
info = raw.info
raw_1 = mne.io.RawArray(raw_1_avg, info)

Control group data (Online control group data eye closed)

In [ ]:
# PRE VS POST COMPARISON
eye_close_6 = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', 'Active', 'PreetiSingh', 'Post', '20230826225729_PreetiSingh_08.09.23_20_Eye Close.easy' )
eye_close_7 = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', 'Active', 'Hemlata', 'Post', '20231019110530_Hemlata_19.10.23_19A_Eye Close.easy') 
eye_close_8 = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', 'Active', 'VinodKumarSharma', 'Post', '20230901112451_VinodKumarSharma_11.10.23_20_Eye Close.easy' ) 
# eye_close_9 = '\\Depression-Sample-dataset-AIIMS\\Sham\\JitenderKumar\\post\\' + '20230827174115_JitenderKumar_13.09.23_20_Eye\ Close'
# eye_close_10 = '\\Depression-Sample-dataset-AIIMS\\Sham\\SeemaKumari\\post\\' + '20230829172634_SeemaKumari_23.9.23_20_Eye\ Close'
eye_close_control = [eye_close_6, eye_close_7, eye_close_8]

In [ ]:
# PRE VS POST COMPARISON
raw_2 = []
# folder_path = os.getcwd()+'\\Depression-Sample-dataset-AIIMS\\'
for file_path in eye_close_control:
    raw = data_transformation_easy(file_path)
    # raw = mne.io.read_raw_edf(file, eog=None, misc=None, stim_channel='auto', exclude=(), infer_types=False, include=None, preload=True, units=None, encoding='utf8', verbose=None)
    # By default read_raw_edf convert units to V
    raw = g1_preprocess(raw)
    raw_2.append(raw.get_data())

In [ ]:
# PRE VS POST COMPARISON
# M1 : Averaging using mean:
raw_2_avg = np.mean(raw_2, axis=0)
print(raw_2_avg)

del raw_2[:]
del raw_2
del eye_close_control[:]
del eye_close_control

In [ ]:
# PRE VS POST COMPARISON
info = raw.info
raw_2 = mne.io.RawArray(raw_2_avg, info)

In [ ]:
# # Find control indexes
# file_path = os.getcwd() + '\\Depression-Sample-dataset-AIIMS\\' + 'Control\\' + 'derivatives\\cleaned_epochs\\'
# control_indexes = []
# for i in range(0, 111):
#     eeg_file_epoch = file_path + f'sub-{i+1:03d}\\ses-t1\\eeg\\sub-{i+1:03d}_ses-t1_task-resteyesc_desc-epochs_eeg.set'
#     epochs_data = mne.io.read_epochs_eeglab(eeg_file_epoch, events=None, event_id=None, eog=(), uint16_codec=None, montage_units='mm', verbose=None)
#     if len(epochs_data.get_data()) == 55:
#         control_indexes.append(i)

In [ ]:
# file_path = os.getcwd() + '\\Depression-Sample-dataset-AIIMS\\' + 'Control\\'
# raw_2 = []
# for i in control_indexes:
#     eeg_file_raw = file_path + f'sub-{i+1:03d}\\ses-t1\\eeg\\sub-{i+1:03d}_ses-t1_task-resteyesc_eeg.edf'
#     raw = mne.io.read_raw_edf(eeg_file_raw, eog=None, misc=None, stim_channel='auto', exclude=(), infer_types=False, include=None, preload=False, units=None, encoding='utf8', verbose=None)
#     # By default read_raw_edf convert units to V
#     raw = control_preprocessing(raw, raw_1)
#     raw_2.append(raw.get_data())

In [ ]:
# del control_indexes[:]
# del control_indexes

In [ ]:
# raw_2_avg = np.mean(raw_2, axis=0)

# del raw_2[:]
# del raw_2

In [ ]:
# info = raw.info
# raw_2 = mne.io.RawArray(raw_2_avg, info)

Saving Raw MDD and control data

In [ ]:
raw_1.save("loadData/rawMDDpre3Active.fif", overwrite=True)
raw_2.save("loadData/rawMDDpost3Active.fif", overwrite=True)

### Epoching

In [ ]:
duration = 4.0
overlap = 2.0 

samples_per_epoch = int(duration * SFREQ)
samples_per_overlap = int(overlap * SFREQ)

# Manually created events
start, stop = 0, samples_per_epoch
events = []
while stop <= len(raw_1):
    events.append([start, 0, 1]) 
    start += samples_per_overlap
    stop += samples_per_overlap

events = np.array(events)

3.1 Segmenting epochs for MDD

In [ ]:
epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
raw_1.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
epochs_1.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

3.2 Segmenting epochs for control

In [ ]:
epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
raw_2.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
epochs_2.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

In [ ]:
# file_path = os.getcwd() + '\\Depression-Sample-dataset-AIIMS\\' + 'Control\\' + 'derivatives\\cleaned_epochs\\'
# epochs_2 = []
# for i in range(0, 111):
#     eeg_file_epoch = file_path + f'sub-{i+1:03d}\\ses-t1\\eeg\\sub-{i+1:03d}_ses-t1_task-resteyesc_desc-epochs_eeg.set'
#     epochs_data = mne.io.read_epochs_eeglab(eeg_file_epoch, events=None, event_id=None, eog=(), uint16_codec=None, montage_units='mm', verbose=None)
#     if len(epochs_data.get_data()) == 55:
#         cleaned_epoch = control_preprocessing(epochs_data, epochs_1, crop=False)
#         epochs_2.append(cleaned_epoch)

In [ ]:
# epochs_2_avg = np.mean(epochs_2, axis=0)
# print(epochs_2_avg)

# del epochs_2[:]
# del epochs_2

In [ ]:
# info = cleaned_epoch.info
# epochs_2 = mne.EpochsArray(epochs_2_avg, info)
# epochs_2.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=None, scalings=SCALINGS)

Saving epoched raw and control data

In [ ]:
# epochs_1.save("loadData/epochsMDDpre3Active.fif", overwrite=True)
# epochs_2.save("loadData/epochsMDDpost3Active.fif", overwrite=True)

### 1. PSD graph plot between MDD and Control group

For resting state EEG channel, analyzing the overall average power across channels provides a holistic view of the brain's activity without emphasizing the specificity of individual channels.

In [ ]:
# def wavelet_plot(raw1, raw2, title):
#     '''
#     signal ----> breaks into frequency components/scales ----> WT computes coeffcients that represent signal's content 
#     at a specific scale ----->coeffs at lower level represent lower frequency components
     
#     Scales vs frequencies - lower scale = higher frequency (fine details)
#     larger coefficient represents higher energy at a sepcific scale
#     peaks in scale vs time plot = where energy is concentrated
#     broad distribution across scales = mix of frequencies

#     Find frequency component of interest 
#     Step1: Look for peaks (dominant frequencies)
#     Step2: Convert scale to frequency using frequency = SFREQ/(2**scale)
#     Step 3: Compare across time
#     Step 4: Statistical analysis

#     '''
#     # Average channel wavelet transform
#     raw1_avg = np.mean(raw1.get_data(), axis=0)
#     raw2_avg = np.mean(raw2.get_data(), axis=0)

#     # Individual channels wavelet transform
#     t = np.arange(0, 150, 1/SFREQ)
#     ts = t[:-1]
    
#     wavelet = 'db13'
#     level = 4 # level of decomposition based on your signal characteristics
 
#     coeffs1 = pywt.wavedec(raw1_avg, wavelet, level=level)    
#     coeffs2 = pywt.wavedec(raw2_avg, wavelet, level=level)

#     plt.figure(figsize=(10, 6))
#     for i in range(level+1):
#         if i==0:
#             plt.plot(t, raw1_avg, label='MDD raw')
#         plt.plot(t, pywt.upcoef('a', coeffs1[i], wavelet, level=level)[:len(t)], label=f'MDD - Level {i}')
#         if i==0:
#             plt.plot(t, raw2_avg, label='Control raw')
#         plt.plot(t, pywt.upcoef('a', coeffs2[i], wavelet, level=level)[:len(t)], label=f'Control - Level {i}')
#         plt.title('DWT')
#         plt.xlabel('Time')
#         plt.ylabel('Amplitude')
#         plt.legend()
#         plt.show()

# wavelet_plot(raw_1, raw_2, "gjhggj")

In [21]:
# Amplitude plots between both groups
epochs_1_avg = epochs_1['1'].average()
epochs_2_avg = epochs_2['1'].average()
# evokeds = dict(epochs_1=epochs_1)
fig_MDD = epochs_1_avg.plot(titles=GROUP1, time_unit = 'ms')
fig_control = epochs_2_avg.plot(titles=GROUP2, time_unit = 'ms')

1.1 PSD using plot_psd

In [22]:
fig, ax = plt.subplots()
#plot_psd uses welch method for continuous data
raw_1.plot_psd(picks=raw_1.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='blue', dB=False)
raw_2.plot_psd(picks=raw_2.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='red', dB=False)

# dB = True plots PSD in decibels (logarithmic)
# different n_fft compared to psd_array_welch
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power/Frequency (microV^2/Hz)')
ax.set_title('Power Spectral Density Comparison')

ax.text(0.8, 0.9, GROUP1, color='blue', transform=ax.transAxes)
ax.text(0.8, 0.85, GROUP2, color='red', transform=ax.transAxes)

plt.ylim(0, 5)
plt.show()

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


1.2 Average of power using welch method

In [23]:
labels = np.arange(0, FMAX)
# welch always gives positive psd
psd_1, frequencies = mne.time_frequency.psd_array_welch(raw_1.get_data(), fmin=FMIN, fmax=FMAX, sfreq=SFREQ)
psd_2, frequencies = mne.time_frequency.psd_array_welch(raw_2.get_data(), fmin=FMIN, fmax=FMAX, sfreq=SFREQ)
avg_abs_power_1, avg_abs_power_2 = np.mean(psd_1, axis=0), np.mean(psd_2, axis=0) 

plt.figure()
plt.plot(frequencies, avg_abs_power_1, color='blue', label=GROUP1)
plt.plot(frequencies, avg_abs_power_2, color='red', label=GROUP2)
plt.title('Welch - Average Power')
plt.xlabel('Frequency (Hz)')
plt.xticks(labels, labels, rotation ='vertical') 
plt.ylabel('Power (V^2/Hz)')
plt.legend()
plt.show()

Effective window size : 0.512 (s)


Effective window size : 0.512 (s)


1.3 Statistical testing between psds of raw

1.3.1 Plotting Histogram

Shows if the data is bell shaped, skewed (positive or negative), uniform etc. Central tendency - highest peak in histogram. Spread - width of distribution

In [24]:
g1_avg = np.mean(raw_1.get_data(), axis=0)
plt.hist(g1_avg, bins='auto', alpha=0.7, edgecolor='black')
plt.title(f'Histogram of {GROUP1} Data - averaged across all channels')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()

In [25]:
g2_avg = np.mean(raw_2.get_data(), axis=0)
plt.hist(g2_avg, bins='auto', alpha=0.7, edgecolor='black')
plt.title(f'Histogram of {GROUP2} Data - averaged across all channels')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()

In [26]:
print(f"{GROUP1} mean: ", np.mean(g1_avg, axis=0))
print(f"{GROUP2} mean: ", np.mean(g2_avg, axis=0))

PRE-ACTIVE mean:  5.74804513030665e-15
POST-ACTIVE mean:  -9.434248663081896e-15


1.3.2 Calculate variance of both groups

In [27]:
variance_1 = np.var(psd_1, ddof=1) 
variance_2 = np.var(psd_2, ddof=1)
print(f"{GROUP1} variance: ", variance_1)
print(f"{GROUP2} variance: ", variance_2)

PRE-ACTIVE variance:  4.393636051591881e-23
POST-ACTIVE variance:  1.146915391749627e-22


1.3.3 Find differences between the means of two independent groups

For example, null hypothesis: there is no significant difference between groups (ie means are equal).
alternative hypothesis: there is a significant difference between groups
t-test : compares the differences between group means to expected variability in data

In [28]:
p_values = np.zeros(32)
t = np.zeros(32)
for i in range(32):
    t[i], p_values[i] = ttest_ind(psd_1[i, :], psd_2[i, :])
# independent t-test comparison : equal_var = True
# welch's t-test comparison: equal_var = False
print("Raw Data PSD Comparison Results:")
print("T-statistic:", t)
print("P-values:", p_values) # probability of getting t-statistic >= what we got

Raw Data PSD Comparison Results:
T-statistic: [-0.27304604 -0.03453517 -0.21751426 -1.00135135  0.1310346   0.16661762
  0.53889779  0.06025445  0.11036171  0.26668035 -0.16806577 -0.39817457
  0.87833595  0.63308663 -0.29067226 -0.21898807  1.30634713 -0.8337382
  1.44027416  1.18135668  0.28608419 -0.80793568  0.84484441 -1.06582949
 -0.92871372  0.23145593  0.32488679  0.07026924 -0.10960572  1.54623124
  0.4692573   1.12480499]
P-values: [0.78609533 0.9726066  0.82881315 0.3221323  0.89634524 0.86843427
 0.59267373 0.95222583 0.91262438 0.79096116 0.86730169 0.69242703
 0.38453277 0.52995398 0.7726677  0.827672   0.19822231 0.40893198
 0.15686793 0.24380712 0.7761563  0.42347425 0.4027683  0.29231451
 0.35810455 0.81803364 0.74680649 0.94429778 0.91322045 0.12921175
 0.64120279 0.2667704 ]


In [29]:
t_statistic, p_values = ttest_ind(np.mean(psd_1, axis=0), np.mean(psd_2, axis=0), equal_var=False)
# independent t-test comparison : equal_var = True
# welch's t-test comparison: equal_var = False
print("Raw Data PSD Comparison Results:")
print("T-statistic:", t_statistic)
print("P-values:", p_values) # probability of getting t-statistic >= what we got

Raw Data PSD Comparison Results:
T-statistic: -0.06768540561794865
P-values: 0.9463921453925532


If p value < 0.005, we reject the null hypothesis

### 2. PLI and construction of brain function matrices

In [30]:
freq_bands = [(0.01, 4), (4,8), (8, 10), (10, 13), (13, 15), (15, 22)]
mapping = {0: 'delta', 1: 'theta', 2: 'alpha1', 3:'alpha2', 4:'beta1', 5:'beta2'}
connectivity = {}

In [31]:
# pli method always gives positive correlations
def connectivity_matrix(epochs, i):
    return spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=SFREQ,
    fmin=freq_bands[i][0], fmax=freq_bands[i][1], faverage=True, n_jobs=1)


In [32]:
n_channels = len(raw_1.info['ch_names'])
for i in range(len(freq_bands)):
    con = connectivity_matrix(epochs_1, i)
    connectivity[f'{GROUP1}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    con = connectivity_matrix(epochs_2, i)
    connectivity[f'{GROUP2}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    # Make matrix symmetric
    for row in range(n_channels):
        for col in range(n_channels):
            connectivity[f'{GROUP1}-{mapping[i]}'][row][col] = connectivity[f'{GROUP1}-{mapping[i]}'][col][row]
            connectivity[f'{GROUP2}-{mapping[i]}'][row][col] = connectivity[f'{GROUP2}-{mapping[i]}'][col][row]
    

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 0.2Hz..4.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11


/tmp/ipykernel_10587/1345682627.py:3: RuntimeWarning: There were no Annotations stored in <EpochsFIF |  73 events (all good), 0 – 4 s, baseline off, ~54 kB, data not loaded,
 '1': 73>, so metadata was not modified.
  return spectral_connectivity_epochs(
/tmp/ipykernel_10587/1345682627.py:3: RuntimeWarning: fmin=0.010 Hz corresponds to 0.040 < 5 cycles based on the epoch length 4.002 sec, need at least 500.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  return spectral_connectivity_epochs(


    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36


/tmp/ipykernel_10587/1345682627.py:3: RuntimeWarning: There were no Annotations stored in <EpochsFIF |  73 events (all good), 0 – 4 s, baseline off, ~54 kB, data not loaded,
 '1': 73>, so metadata was not modified.
  return spectral_connectivity_epochs(
/tmp/ipykernel_10587/1345682627.py:3: RuntimeWarning: fmin=0.010 Hz corresponds to 0.040 < 5 cycles based on the epoch length 4.002 sec, need at least 500.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  return spectral_connectivity_epochs(


    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
  

...

    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    c

/tmp/ipykernel_10587/1345682627.py:3: RuntimeWarning: There were no Annotations stored in <EpochsFIF |  73 events (all good), 0 – 4 s, baseline off, ~54 kB, data not loaded,
 '1': 73>, so metadata was not modified.
  return spectral_connectivity_epochs(


    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    c

/tmp/ipykernel_10587/1345682627.py:3: RuntimeWarning: There were no Annotations stored in <EpochsFIF |  73 events (all good), 0 – 4 s, baseline off, ~54 kB, data not loaded,
 '1': 73>, so metadata was not modified.
  return spectral_connectivity_epochs(


    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epoch 15
    computing connectivity for epoch 16
    computing connectivity for epoch 17
    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
   

In [33]:
print((connectivity.keys()))

dict_keys(['PRE-ACTIVE-delta', 'POST-ACTIVE-delta', 'PRE-ACTIVE-theta', 'POST-ACTIVE-theta', 'PRE-ACTIVE-alpha1', 'POST-ACTIVE-alpha1', 'PRE-ACTIVE-alpha2', 'POST-ACTIVE-alpha2', 'PRE-ACTIVE-beta1', 'POST-ACTIVE-beta1', 'PRE-ACTIVE-beta2', 'POST-ACTIVE-beta2'])


In [34]:
# # ALTERNATIVE: PLI calculation in Raw Python

# def calculate_pli(data):
#     # data = EEG data of channel Ci and Cj for 1 epoch (2000 data points (4s X 500fs))
#     analytic_signal = hilbert(data)
#     instantaneous_phase = np.angle(analytic_signal)
#     return np.abs(np.mean(np.sign(np.diff(instantaneous_phase, axis=0))))

# fun_connectivity = []
# for k in range(len(epochs)):
#     pli_matrix = [[0 for i in range(len(raw_cleaned.info['ch_names']))] for j in range(len(raw_cleaned.info['ch_names']))]
#     for i in range(len(raw_cleaned.info['ch_names'])):
#         for j in range(i+1, len(raw_cleaned.info['ch_names'])):
#             pli_matrix[i][j] = pli_matrix[j][i] = calculate_pli([epochs[k].get_data(fmin = freq_bands[0][0], fmax=freq_bands[-1][-1])[0][i], epochs[k].get_data(fmin = freq_bands[0][0], fmax=freq_bands[-1][-1])[0][j]])

#     fun_connectivity.append(pli_matrix)
# conn = np.mean(fun_connectivity, axis=0)

### 3. Thresholding - M1 -  Small World Index

In [35]:
# Preserves small-world properties in both groups to ensure that any observed differences in connectivity are not biased by the thresholding method.
# Preserving these properties ensures that information can be transmitted quickly and effectively across the network.

In [36]:
optimal_threshold = {}
edges_1 = {}
edges_2 = {}
# Generating 50 random networks with same number of vertices and edges 
random_networks = [nx.gnm_random_graph(n_channels, 496) for _ in range(50)]
thresholds = np.arange(0.1, 0.9, 0.002)

for i in range(len(freq_bands)):
    edges_1[mapping[i]] = []
    edges_2[mapping[i]] = []
    ratios = []
    for threshold in thresholds:
        # TODO: Cross check should we ignore thresholds which result in matrix to be not connected ?
        Lw_1_binarized = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > threshold))[0]
        Cw_1_binarized = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > threshold))[3]

        Lw_2_binarized = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > threshold))[0]
        Cw_2_binarized = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > threshold))[3]

        swi_ratio_binarized =  (Cw_1_binarized*Lw_2_binarized)/(Lw_1_binarized*Cw_2_binarized) if Lw_1_binarized and Cw_2_binarized else 0
        ratios.append(swi_ratio_binarized)

    t_stat, p_value = ttest_ind(np.mean(connectivity[f'{GROUP1}-{mapping[i]}'], axis=0), np.mean(connectivity[f'{GROUP2}-{mapping[i]}'], axis=0)) 
    if p_value < 0.005: # Reject the null hypotheses
        optimal_threshold[mapping[i]] = thresholds[np.argmax(np.abs(ratios))] #TODO: ratios - swi_ratio ? OR remove swi_ratio and related code above
        Crand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[3] for G in random_networks])
        Lrand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[0] for G in random_networks])

        Lw_1_binarized = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]))[0]
        Cw_1_binarized = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]))[3]

        Lw_2_binarized = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]))[0]
        Cw_2_binarized = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]))[3]

        swi_binarized_1 =  (Cw_1_binarized/Crand)/(Lw_1_binarized/Lrand)
        swi_binarized_2 = (Cw_2_binarized/Crand)/(Lw_2_binarized/Lrand)
        print(swi_binarized_1, "Cw1 ", Cw_1_binarized, "Lw1 ", Lw_1_binarized)
        print(swi_binarized_2, "Cw2 ", Cw_2_binarized, "Lw2 ", Lw_2_binarized)  

        edges_1[mapping[i]].append(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]).number_of_edges())
        edges_2[mapping[i]].append(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]).number_of_edges())

        print(f"{mapping[i]} - Optimal Threshold: {optimal_threshold[mapping[i]]} (Significant)", p_value)

    # plt.plot(thresholds, ratios, label='Threshold Ratios')
    # plt.axhline(swi_ratio, color='red', linestyle='--', label=' SWI ratio (MDD/Control)')
    # plt.xlabel(f'Threshold-{mapping[i]}')
    # plt.ylabel('Ratio')
    # plt.legend()
    # plt.show()

0.693393438150762 Cw1  0.8206087665211639 Lw1  1.1834677419354838
0.878539103750922 Cw2  0.9387615423144934 Lw2  1.0685483870967742
delta - Optimal Threshold: 0.1 (Significant) 2.409809725245008e-12
0.2830188679245283 Cw1  0.047619047619047616 Lw1  0.16825396825396824
0.1336432771617957 Cw2  0.09124609957943292 Lw2  0.6827586206896551
theta - Optimal Threshold: 0.6660000000000005 (Significant) 6.4050075170624085e-12
0.2873103190846081 Cw1  0.2459129182702667 Lw1  0.8559139784946237
0.3311533099912656 Cw2  0.5100829210349332 Lw2  1.5403225806451613
alpha1 - Optimal Threshold: 0.4320000000000003 (Significant) 2.3706395712621753e-08
0.09543804601944138 Cw1  0.01609347442680776 Lw1  0.16862745098039214
0.038568778419393525 Cw2  0.033633156966490295 Lw2  0.8720306513409962
alpha2 - Optimal Threshold: 0.7540000000000006 (Significant) 0.0004624686935601696
0.02329650092081031 Cw1  0.003042328042328042 Lw1  0.1305916305916306
0.004956427015250545 Cw2  0.00196969696969697 Lw2  0.397402597402597

In [ ]:
# for i in range(len(freq_bands)):
#     optimal_threshold[mapping[i]]  -= 0.1
# print(optimal_threshold)

### M2 - Community structure

In [ ]:
from networkx.algorithms.community import greedy_modularity_communities
import networkx as nx
# Calculate the participation coefficient for each node
def participation_coefficient(node, modularity_matrix):
    k_in = np.sum(modularity_matrix[node, :])
    k_out = np.sum(modularity_matrix) - k_in
    q = modularity_matrix.shape[0]
    pc = 1 - ((k_in / q) ** 2 + (k_out / q) ** 2)
    return pc

def community_structure(G):
    # Compute community structure (Louvain algorithm)
    partition = list(nx.algorithms.community.modularity_max.greedy_modularity_communities(G)[0])
    # pi_scores = nx.algorithms.community.indicators.participation(G, partition)(G)

    print(nx.modularity_matrix(G))

    # plot network with community structure 
    # pos = nx.spring_layout(G)
    # colors = [partition[node] for node in G.nodes]
    # plt.figure(figsize=(10, 8))
    # nx.draw(G, pos, node_color=colors, with_labels=True, cmap=plt.cm.Paired, node_size=300)
    # plt.title("Community structure")
    # plt.show()

    num_nodes = len(G.nodes)
    mod_matrix = np.zeros((num_nodes, num_nodes))

    # Find modularity matrix
    for i, j, _ in G.edges(data=True):
        mod_matrix[i][j] = 1 if partition[i] == partition[j] else 0

    pi_values = np.zeros(num_nodes)
    for node in range(num_nodes):
        pi_values[node] = participation_coefficient(node, mod_matrix)

    # Rank-order the PI scores (lowest to highest)
    ranked_pi = np.argsort(pi_values) + 1
    return ranked_pi

In [22]:
for i in range(len(freq_bands)):
    # Values ranging from -1 to 1
    diff_matrix = nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] - connectivity[f'{GROUP2}-{mapping[i]}'])
    print(np.min(connectivity[f'{GROUP1}-{mapping[i]}'] - connectivity[f'{GROUP2}-{mapping[i]}']), np.max(connectivity[f'{GROUP1}-{mapping[i]}'] - connectivity[f'{GROUP2}-{mapping[i]}']))
    ranked_pi = community_structure(diff_matrix)

-0.4651826484018265 0.3002758751902588
[[-0.96875  0.03125  0.03125 ...  0.03125  0.03125  0.03125]
 [ 0.03125 -0.96875  0.03125 ...  0.03125  0.03125  0.03125]
 [ 0.03125  0.03125 -0.96875 ...  0.03125  0.03125  0.03125]
 ...
 [ 0.03125  0.03125  0.03125 ... -0.96875  0.03125  0.03125]
 [ 0.03125  0.03125  0.03125 ...  0.03125 -0.96875  0.03125]
 [ 0.03125  0.03125  0.03125 ...  0.03125  0.03125 -0.96875]]
-0.7693588280060883 0.48185407153729076
[[-0.96875  0.03125  0.03125 ...  0.03125  0.03125  0.03125]
 [ 0.03125 -0.96875  0.03125 ...  0.03125  0.03125  0.03125]
 [ 0.03125  0.03125 -0.96875 ...  0.03125  0.03125  0.03125]
 ...
 [ 0.03125  0.03125  0.03125 ... -0.96875  0.03125  0.03125]
 [ 0.03125  0.03125  0.03125 ...  0.03125 -0.96875  0.03125]
 [ 0.03125  0.03125  0.03125 ...  0.03125  0.03125 -0.96875]]
-0.7593226788432268 0.6359874429223745
[[-0.96875  0.03125  0.03125 ...  0.03125  0.03125  0.03125]
 [ 0.03125 -0.96875  0.03125 ...  0.03125  0.03125  0.03125]
 [ 0.03125  0.03

### M3 - Threshold using saturation

In [23]:
for i in range(len(freq_bands)):
    plt.plot(thresholds, edges_MDD[mapping[i]])
    plt.plot(thresholds, edges_control[mapping[i]])
    plt.title(f'{mapping[i]} Threshold comparison (blue - MDD, red-Control)')
    plt.xlabel('Thresholds')
    plt.ylabel('Number of edges')
    plt.show()

In [ ]:
# # To remove:
# optimal_threshold['delta'] = 0.442
# optimal_threshold['theta'] =  0.529
# optimal_threshold['alpha1'] = 0.786
# optimal_threshold['alpha2'] = 0.741

### M4 - ML algorithm

In [ ]:
# raw_1.info

### 4. Binarization on functional connectivity matrices

In [37]:
del freq_bands[-1]

In [38]:
binarized_matrix = {}
for i in range(len(freq_bands)):
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]
print(binarized_matrix.keys())

dict_keys(['PRE-ACTIVE-delta', 'POST-ACTIVE-delta', 'PRE-ACTIVE-theta', 'POST-ACTIVE-theta', 'PRE-ACTIVE-alpha1', 'POST-ACTIVE-alpha1', 'PRE-ACTIVE-alpha2', 'POST-ACTIVE-alpha2', 'PRE-ACTIVE-beta1', 'POST-ACTIVE-beta1'])


Find number of links in each graph

In [39]:
for i in range(len(freq_bands)):
    links_MDD = 0
    links_control = 0

    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                links_MDD+=1
            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                links_control+=1

    print(f'{GROUP1}-{mapping[i]}-number of links: K=', links_MDD)
    print(f'{GROUP2}-{mapping[i]}-number of links: K=', links_control)

PRE-ACTIVE-delta-number of links: K= 405
POST-ACTIVE-delta-number of links: K= 462
PRE-ACTIVE-theta-number of links: K= 13
POST-ACTIVE-theta-number of links: K= 108
PRE-ACTIVE-alpha1-number of links: K= 152
POST-ACTIVE-alpha1-number of links: K= 240
PRE-ACTIVE-alpha2-number of links: K= 24
POST-ACTIVE-alpha2-number of links: K= 57
PRE-ACTIVE-beta1-number of links: K= 14
POST-ACTIVE-beta1-number of links: K= 36


In [40]:
# Yellow - Higher synchronisation
for i in range(len(binarized_matrix)//2):
    # i = 0, 1, 2
    plt.figure(figsize=(12, 4))
    for j in range(2):
        # j = 0, 1
        plt.subplot(1, 2, j+1)
        plt.imshow(list(binarized_matrix.values())[2*i+j], interpolation='none')
        plt.title(f'Functional connectivity - {list(binarized_matrix)[2*i+j]} - Visualization')
        plt.xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
        plt.yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

Asymmetry in  each band

In [41]:
left_region = ['Fp1', 'AF3', 'PO3','O1', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']
right_region = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2', 'FC2', 'CP2', 'O2', 'PO4']

for i in range(len(freq_bands)):
    lr = 0 
    rr = 0
    lrc = 0
    rrc = 0
    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lr += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rr += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lrc += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rrc += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

    print(f"{GROUP1}: ",mapping[i], "left region: ", lr, "right region: ", rr, "proportion: ", lr/rr)
    print(f"{GROUP2}: ", mapping[i], "left region: ", lrc, "right region: ", rrc, "proportion: ", lrc/rrc)

PRE-ACTIVE:  delta left region:  292 right region:  268 proportion:  1.0895522388059702
POST-ACTIVE:  delta left region:  316 right region:  327 proportion:  0.9663608562691132
PRE-ACTIVE:  theta left region:  9 right region:  4 proportion:  2.25
POST-ACTIVE:  theta left region:  78 right region:  66 proportion:  1.1818181818181819
PRE-ACTIVE:  alpha1 left region:  116 right region:  79 proportion:  1.4683544303797469
POST-ACTIVE:  alpha1 left region:  176 right region:  160 proportion:  1.1
PRE-ACTIVE:  alpha2 left region:  19 right region:  10 proportion:  1.9
POST-ACTIVE:  alpha2 left region:  42 right region:  36 proportion:  1.1666666666666667
PRE-ACTIVE:  beta1 left region:  10 right region:  6 proportion:  1.6666666666666667
POST-ACTIVE:  beta1 left region:  29 right region:  19 proportion:  1.5263157894736843


### 6. Difference matrix

In [42]:
def return_index(channels_list, raw):
    indexes = []
    for channel in channels_list:
        for i in range(len(raw.info['ch_names'])):
            if raw.info['ch_names'][i]==channel:
                indexes.append(i)
    return(indexes)

In [43]:
central_channels = return_index(['Pz', 'Oz', 'Fz', 'Cz'], raw_1)
central_channels
raw_1.info['bads'] = []

In [ ]:
# ALL_CHANNELS = np.array(['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2',
#        'Fp1', 'AF3', 'Fz', 'FC2', 'Cz', 'CP2', 'PO3', 'O1', 'Oz', 'O2',
#        'PO4', 'Pz', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5',
#        'T7', 'P7'])
# def set_montage(raw):
#     mont1020 = mne.channels.make_standard_montage('standard_1020')
#     ind = [i for (i, channel) in enumerate(mont1020.ch_names) if channel in ALL_CHANNELS]
#     mont1020_new = mont1020.copy()
#     mont1020_new.ch_names = [mont1020.ch_names[x] for x in ind]
#     kept_channel_info = [mont1020.dig[x+3] for x in ind]
#     mont1020_new.dig = mont1020.dig[0:3]+kept_channel_info
#     print("sjkghdkgu", mont1020_new)
#     return raw.set_montage(mont1020_new)

In [45]:
difference_matrix = {}
for i in range(len(freq_bands)):
    difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
    # difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) 
    for k in range(n_channels):
        for l in range(n_channels):
            if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
                difference_matrix[mapping[i]][k][l] = 0

    # Yellow - Higher synchronisation
    # plt.imshow(difference_matrix[mapping[i]], interpolation='none')
    # plt.title(f'Functional connectivity - {[mapping[i]]} - Visualization')
    # plt.xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
    # plt.yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
    # plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    # plt.show()

    plot_sensors_connectivity(
        raw_1.info,
        difference_matrix[mapping[i]],
        cbar_label=f'{mapping[i]}-Connectivity',
        )

Using notebook 3d backend.



In [48]:
conn_measure = ConnectivityMeasure(kind='coherence', sfreq=raw.info['sfreq'])

# Compute connectivity for each frequency band
connectivity_matrices, freqs, times, n_epochs, n_tapers = conn_measure.fit_transform([raw])

for i, freq_band in enumerate(freq_bands):
    plt.imshow(connectivity_matrices[0, i, :, :], cmap='viridis', aspect='auto', extent=[times[0], times[-1], freqs[0], freqs[-1]])
    plt.title(f'Connectivity Matrix ({freq_band[0]}-{freq_band[1]} Hz)')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.colorbar()
    plt.show()

ModuleNotFoundError: No module named 'nilearn'

In [42]:
selected_channels = {}
for i in range(len(freq_bands)):
    selected_channels[mapping[i]] = []
    for ch in range(32):
        # TODO: instead of number of edges wise, we can explore other algorithm
        if len(nx.from_numpy_array(difference_matrix[mapping[i]]).edges(ch)) > 8:
            selected_channels[mapping[i]].append(raw_1.info['ch_names'][ch])
print(selected_channels)

{'delta': ['C3'], 'theta': ['O2', 'FC1', 'C3', 'T7'], 'alpha1': ['C3'], 'alpha2': []}


Create dictionary of Regions in brain vs channels

In [29]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['FP1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'FP2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [30]:
# for i in range(len(freq_bands)):
selected_regions = {}
# selected_regions['alpha2'] = ['RPO', 'LT', 'LF', 'LC']
# selected_regions['alpha1'] = ['LT', 'LC', 'LF', 'RPO'] # 1 in RPO
# selected_regions['theta'] = ['LC', 'LF'] # 1 in LC and 1 in LF
# selected_regions['delta'] =  ['LF', 'LC', 'RF', 'RC'] # 1-1 in 'LC', 'RF', 'RC'
selected_regions['delta'] =  ['LF', 'LC', 'RT']
selected_regions['theta'] = ['LPO', 'LC', 'RPO']
selected_regions['alpha1'] = ['LPO', 'RPO']
selected_regions['alpha2'] = ['LC', 'RPO']

### 7. Network Metrics for selected regions for selected bands


Boxplot helps in reading median, interquartile range and outliers
Box : middle 50% of all data lies (Interquartile range)
Lower end: 1st quartile
Upper end: 3rd quartile
Solid line: Median
Dashed line: Mean
T shaped whiskers: Last point 1.5 times the interquartile range (max and min without outliers)
Points: Further out are outliers

In [31]:
group_measures = ['Lw', 'NBC', 'Eglo', 'CC', 'Eloc']

In [32]:
# def calculate_avergae_components(G):
#     connected_components = list(nx.connected_components(G))
#     component_avg_lengths = []
#     component_nbc_values = []
#     component_eglo_values = []
#     component_cc_values = []
#     component_eloc_values = []

#     for component in connected_components:
#         subgraph = G.subgraph(component)
#         component_avg_lengths.append(nx.average_shortest_path_length(subgraph))
#         component_nbc_values.append(nx.betweenness_centrality(subgraph))
#         component_eglo_values.append(nx.global_efficiency(subgraph))
#         component_cc_values.append(nx.clustering(subgraph))
#         component_eloc_values.append(nx.local_efficiency(subgraph))

#     merged_nbc_values = {}
#     for dnbc in component_nbc_values:
#         merged_nbc_values.update(dnbc)
#     merged_nbc_values = dict(sorted(merged_nbc_values.items(), key=lambda item: item[0]))
#     merged_cc_values = {}
#     for dcc in component_cc_values:
#         merged_cc_values.update(dcc)
#     merged_cc_values = dict(sorted(merged_cc_values.items(), key=lambda item: item[0]))

#     overall_avg_length = sum(component_avg_lengths) / len(component_avg_lengths)
#     overall_eglo_values = sum(component_eglo_values) / len(component_eglo_values)
#     overall_eloc_values = sum(component_eloc_values) / len(component_eloc_values)

#     return [overall_avg_length, merged_nbc_values, overall_eglo_values, merged_cc_values, overall_eloc_values]

In [38]:
# Integration metrics - Lavg, Eglo 
# Segregation metrics - CC, Eloc, Nbc
import seaborn as sns
for i in range(len(freq_bands)):
    selected_region = selected_regions[mapping[i]]

    if selected_region == []:
        continue

    # con_MDD = connectivity_matrix(epochs_1, i)
    # con_control = connectivity_matrix(epochs_2, i)
    G_MDD = nx.from_numpy_array(binarized_matrix[f'{GROUP1}-{mapping[i]}'])
    G_Control = nx.from_numpy_array(binarized_matrix[f'{GROUP2}-{mapping[i]}'])

    for region in selected_region:
        channels_list = regionvschannel[region]
        G_sub = G_MDD.subgraph(return_index(channels_list, raw_1))
        MDD_list = calculate_avergae_components(G_sub)
        G_sub = G_Control.subgraph(return_index(channels_list, raw_2))
        control_list = calculate_avergae_components(G_sub)

        print(MDD_list, "fdhgjgdfjd", control_list)

        # CHECK HERE if keys in MDD_list[1] matches with keys in control_list[1]
        # and keys in MDD_list[3] matches with keys in control_list[3]
        
        values_MDD = []
        values_control = []
        group_labels_MDD = []
        group_labels_control = [] 

        for _ in range(5):
            print("********", MDD_list)
            if len(MDD_list[_]):
                if type(MDD_list[_][0]) == type(dict()):
                    x = []
                    for __ in range(len(MDD_list[_])):
                        for k,v in MDD_list[_][__].items():
                            x.append(v)
                    values_MDD.append(x)
                else:
                    values_MDD.append(MDD_list[_])
            else:
                values_MDD.append([])
            if (control_list[_]):
                if type(control_list[_][0]) == type(dict()):
                    y = []
                    for __ in range(len(control_list[_])):
                        for k,v in control_list[_][__].items():
                            y.append(v)
                    values_control.append(y)
                else:
                    values_control.append(control_list[_])
            else:
                values_control.append([])
            group_labels_MDD.append(f'{GROUP1}-{group_measures[_]}')
            group_labels_control.append(f'{GROUP2}-{group_measures[_]}')
            
        values_all = values_1 + values_control
        group_labels = group_labels_MDD + group_labels_control

        print("values all", values_all)
        print("group labels", group_labels)

        plt.figure(figsize=(12, 8))
        plt.boxplot(values_all, labels=group_labels, vert=True)
        plt.title(f'Boxplot of Network Metrics for MDD and Control Groups in {mapping[i]} band in {region} region')
        plt.xlabel('Network Metrics')
        plt.ylabel('Values')
        plt.show()

[0.0, {11: 0.0, 26: 0.0, 27: 0.0}, 0.0, 0.0, 0.0] fdhgjgdfjd [0.0, {11: 0.0, 26: 0.0, 27: 0.0}, 0.0, 0.0, 0.0]
******** [0.0, {11: 0.0, 26: 0.0, 27: 0.0}, 0.0, 0.0, 0.0]


TypeError: object of type 'float' has no len()

8. Mark all regions in brain along with Netowrk metrics which are significant

[]


9. Find relationship between "these" Network Metrics and PHQ-9 scores 

10. For further evaluation: RF based on ensemble learning, SVM, KNN, ANN with 10-fold cross validation 